# Gaussian Quadrature for Financial Derivatives

This notebook explores the theoretical foundations and practical implementation of Gaussian Quadrature methods for pricing financial derivatives, specifically focusing on caplet pricing using the Bachelier model.

## Table of Contents
1. [Mathematical Foundations](#Mathematical-Foundations)
2. [Implementation Details](#Implementation-Details)
3. [Numerical Examples](#Numerical-Examples)
4. [Performance Analysis](#Performance-Analysis)

Let's start by importing the necessary libraries:

In [21]:
import sys
import os

# Go up one level from 'notebooks/' to the project root
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import display, HTML
import pandas as pd

# Import quadrature functions
from src.gaussian_quadrature.utils.quadrature import (
    laguerre_function_final,
    compute_gauss_laguerre_points_final,
    generalized_laguerre_quadrature
)

In [22]:
def gauss_laguerre_quadrature_final(f, n_points, alpha=0.0):
    """
    Final version of Gauss-Laguerre quadrature with improved numerical stability.
    
    Parameters:
    f : callable
        The integrand function
    n_points : int
        Number of quadrature points
    alpha : float, optional
        Parameter for generalized Laguerre polynomial

    Returns:
    float
        Approximated integral value for int_0^inf f(x) * x^alpha * e^(-x) dx
    """
    if n_points > 100:
        raise ValueError("n_points > 100 may lead to numerical instability")
    
    # Get Gauss-Laguerre nodes and weights
    nodes, weights = compute_gauss_laguerre_points_final(n_points, alpha)
    
    # Evaluate integrand and handle potential NaN/inf values
    f_vals = jnp.array([f(x) for x in nodes])
    f_vals = jnp.where(jnp.isfinite(f_vals), f_vals, 0.0)
    
    # Use Kahan summation for improved accuracy
    result = 0.0
    correction = 0.0
    for w, v in zip(weights, f_vals):
        term = w * v - correction
        temp = result + term
        correction = (temp - result) - term
        result = temp
    
    return result

## Mathematical Foundations

### Gaussian Quadrature Theory

Gaussian quadrature is a numerical integration technique that approximates the definite integral of a function by a weighted sum:

$$\int_a^b f(x)w(x)dx \approx \sum_{i=1}^n w_i f(x_i)$$

where:
- $w(x)$ is a weight function
- $x_i$ are the quadrature points (nodes)
- $w_i$ are the quadrature weights

### Gauss-Laguerre Quadrature

For our caplet pricing, we use Gauss-Laguerre quadrature, which is specialized for integrals of the form:

$$\int_0^\infty f(x)e^{-x}dx \approx \sum_{i=1}^n w_i f(x_i)$$

The nodes $x_i$ are the roots of the nth Laguerre polynomial $L_n(x)$, and the weights are given by:

$$w_i = \frac{(n!)^2}{x_i[L_{n+1}(x_i)]^2}$$

### Implementation of Laguerre Functions

Let's examine how we compute Laguerre polynomials numerically:

In [23]:
# Generate points for visualization
x = jnp.linspace(0, 10, 100)

# Compute Laguerre polynomials for different orders
results = []
for n in [1, 2, 3, 4, 5, 6]:
    L_n, _ = laguerre_function_final(n, x)
    results.append(go.Scatter(
        x=x,
        y=L_n,
        name=f'L_{n}(x)'
    ))

# Plot results
fig = go.Figure(data=results)
fig.update_layout(
    title='Laguerre Polynomials',
    xaxis_title='x',
    yaxis_title='L_n(x)',
    template='plotly_dark'
)
fig.show()

## Caplet Pricing with Bachelier Model

The Bachelier model for caplet pricing involves computing:

$$V_{caplet} = \tau P(t,T)\mathbb{E}^T_t[(R(T) - K)^+]$$

where:
- $\tau$ is the accrual period
- $P(t,T)$ is the discount factor
- $R(T)$ is the forward rate
- $K$ is the strike rate

Let's implement and compare both pricing methods:

In [24]:
# Example parameters
RtT = 0.05      # Forward rate (5%)
K = 0.04        # Strike rate (4%)
sigma_gtT = 0.01 # Volatility (1%)
tau = 0.5       # Accrual period (6 months)
PtT = 0.98      # Discount factor

# Compare methods with different numbers of quadrature points
n_points_list = [5, 10, 20, 30]
results = []

for n in n_points_list:
    result = compare_pricing_models(RtT, K, sigma_gtT, tau, PtT, n)
    results.append({
        'n_points': n,
        'taylor': result['Taylor Series'],
        'laguerre': result['Laguerre Quadrature']
    })

# Create comparison table
from IPython.display import display, HTML
import pandas as pd

df = pd.DataFrame([
    {
        'Points': r['n_points'],
        'Taylor Price': r['taylor']['Price'],
        'Laguerre Price': r['laguerre']['Price'],
        'Relative Diff (%)': abs(r['taylor']['Price'] - r['laguerre']['Price'])/r['taylor']['Price'] * 100
    } for r in results
])

display(HTML(df.to_html(float_format=lambda x: '%.8f' % x)))

NameError: name 'compare_pricing_models' is not defined

## Numerical Stability Analysis

Our implementation includes several numerical stability improvements:

1. Scaled recurrence relations for Laguerre polynomials
2. Careful handling of exponential terms
3. Adaptive damping in Newton iterations

Let's examine the stability of our quadrature implementation:

In [13]:
import pandas as pd

# Test stability for different numbers of points
def analyze_stability(n_points_list=[5, 10, 20, 30]):
    results = []
    for n in n_points_list:
        nodes, weights = compute_gauss_laguerre_points_final(n)
        results.append({
            'n': n,
            'max_node': float(jnp.max(nodes)),
            'min_weight': float(jnp.min(weights)),
            'max_weight': float(jnp.max(weights)),
            'condition': float(jnp.max(weights)/jnp.min(weights))
        })
    return pd.DataFrame(results)

stability_df = analyze_stability()
display(HTML(stability_df.to_html(float_format=lambda x: '%.2e' % x)))

NameError: name 'HTML' is not defined

## Error Analysis

For validation, let's compute a known integral and analyze the error convergence:

In [ ]:
# Test function with known integral
def test_func(x):
    return x * jnp.exp(-x/2)  # ∫₀^∞ xe^(-3x/2)dx = 4/9

exact = 4.0/9.0
n_points = range(5, 31, 5)
errors = []

for n in n_points:
    result = generalized_laguerre_quadrature(test_func, n)
    rel_error = abs(result - exact)/exact
    errors.append({
        'n_points': n,
        'result': result,
        'rel_error': rel_error
    })

# Plot error convergence
fig = go.Figure(data=go.Scatter(
    x=[e['n_points'] for e in errors],
    y=[e['rel_error'] for e in errors],
    mode='lines+markers'
))

fig.update_layout(
    title='Error Convergence',
    xaxis_title='Number of quadrature points',
    yaxis_title='Relative error',
    yaxis_type='log',
    template='plotly_dark'
)
fig.show()